In [3]:
import cv2
import numpy as np
import random
import noise
import os
from tqdm import tqdm

In [4]:
# Using Cubic Bezier curves to make scratches that look like real life
class Bezier():
    @staticmethod
    def TwoPoints(t, P1, P2):
        """ Returns a point between P1 and P2, parameterized by t. """
        if not isinstance(P1, np.ndarray) or not isinstance(P2, np.ndarray):
            raise TypeError('Points must be an instance of the numpy.ndarray!')
        if not isinstance(t, (int, float)):
            raise TypeError('Parameter t must be an int or float!')
        return (1 - t) * P1 + t * P2

    @staticmethod
    def Points(t, points):
        """ Returns a list of points interpolated by the Bezier process. """
        newpoints = []
        for i1 in range(len(points) - 1):
            newpoints.append(Bezier.TwoPoints(t, points[i1], points[i1 + 1]))
        return newpoints

    @staticmethod
    def Point(t, points):
        """ Returns a point interpolated by the Bezier process. """
        newpoints = points
        while len(newpoints) > 1:
            newpoints = Bezier.Points(t, newpoints)
        return newpoints[0]

    @staticmethod
    def Curve(t_values, points):
        """ Returns points interpolated by the Bezier process. """
        if not hasattr(t_values, '__iter__') or len(t_values) < 1:
            raise TypeError("`t_values` must be an iterable of integers or floats, of length greater than 0.")
        curve = np.empty((0, len(points[0])), dtype=float)
        for t in t_values:
            curve = np.append(curve, [Bezier.Point(t, points)], axis=0)
        return curve

In [11]:
def add_realistic_scratches(image, num_scratches=5):
    scratched_image = image.copy()
    scratch_mask = np.zeros(image.shape[:2], dtype=np.uint8) 
    height, width = scratched_image.shape[:2]

    for _ in range(num_scratches):
        # Randomly select control points for the Bézier curve
        p0 = np.array([random.randint(0, width), random.randint(0, height)])
        p1 = np.array([random.randint(0, width), random.randint(0, height)])
        p2 = np.array([random.randint(0, width), random.randint(0, height)])
        p3 = np.array([random.randint(0, width), random.randint(0, height)])

        # Generate t values for the Bézier curve
        t_values = np.linspace(0, 1, num=100)
        
        # Calculate points on the Bézier curve
        curve_points = Bezier.Curve(t_values, [p0, p1, p2, p3])

        color = (255,255,255)
        
        # Draw the scratch on the image and update the mask
        for i in range(len(curve_points) - 1):
            start_point = tuple(map(int, curve_points[i]))
            end_point = tuple(map(int, curve_points[i + 1]))
            
            # Draw the scratch on the image
            cv2.line(scratched_image, start_point, end_point, color, thickness=random.randint(1, 1))
            
            # Update the mask: Mark the scratch region as white (255)
            cv2.line(scratch_mask, start_point, end_point, 255, thickness=random.randint(1, 1))

    return scratched_image, scratch_mask

In [12]:
def create_random_blob(center, size):
    """Generate a random blob shape centered at `center` with the given `size`."""
    num_points = random.randint(6, 12)  # Number of points for the blob's contour
    angles = np.linspace(0, 2 * np.pi, num_points)
    radii = np.random.uniform(size * 0.5, size)  # Randomize the radius
    points = np.array([
        center[0] + radii * np.cos(angles + random.uniform(0, 0.5)),
        center[1] + radii * np.sin(angles + random.uniform(0, 0.5))
    ]).T.astype(np.int32)

    return points

def add_organic_blobs(image, num_blobs=5):
    damaged_image = image.copy()
    height, width = damaged_image.shape[:2]
    blob_centers = []  # Store centers for potential overlap
    

    for _ in range(num_blobs):
        # Determine the center and size of the blob
        corner_prob = random.randint(0,1)
        if random.random() < corner_prob:  # Probability for corner placement
            # Choose a random corner
            corner = random.choice([(0, 0), (0, height), (width, 0), (width, height)])
            center = (corner[0] + random.randint(20, 20), corner[1] + random.randint(20, 20))
        else:
            # Randomly determine the center within the image
            center = (random.randint(50, width - 50), random.randint(50, height - 50))

        size = random.randint(4, 6)  # Random size for the blob

        # Generate a random blob shape
        blob_shape = create_random_blob(center, size)

        # Draw the blob shape on the image (white color)
        cv2.fillPoly(damaged_image, [blob_shape], color=(255, 255, 255))
        blob_centers.append(center)  # Store the center of the blob

        # Overlap multiple blobs
        for _ in range(random.randint(1, 7)):  # Overlap 1 to 4 additional blobs
            # Pick a random existing blob center to overlap with
            existing_center = random.choice(blob_centers)
            # Offset the overlapping blob's center slightly towards the existing blob
            overlap_center = (
                existing_center[0] + random.randint(-20, 20),
                existing_center[1] + random.randint(-20, 20)
            )
            overlap_size = random.randint(8,10)  # Random size for the overlapping blob
            overlap_shape = create_random_blob(overlap_center, overlap_size)

            # Draw the overlapping blob
            cv2.fillPoly(damaged_image, [overlap_shape], color=(255, 255, 255))

    return damaged_image

In [13]:
def add_realistic_damage(image, num_scratches=5, num_patches=3):
    scratched_image = add_realistic_scratches(image, num_scratches)
    #damaged_image = add_organic_blobs(scratched_image, num_patches)
    return scratched_image

In [14]:
def add_dust_and_grain(image, intensity=0.2):
    # Create random noise (grain)
    noise = np.random.normal(0, 25, image.shape).astype(np.uint8)

    # Add the noise to the image with alpha blending
    noisy_image = cv2.addWeighted(image, 1 - intensity, noise, intensity, 0)

    return noisy_image

In [15]:
def process_images(input_folder, output_folder, output_folder1):
    
    num_scratches = random.randint(4, 7)
    num_patches = random.randint(10, 20)
    
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    if not os.path.exists(output_folder1):
        os.makedirs(output_folder1)
        
    image_files = [f for f in os.listdir(input_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]

    # Loop through each image in the input folder
    for filename in tqdm(image_files, desc="Processing images", unit="image"):
        if filename.endswith(('.png', '.jpg', '.jpeg')):  # Check for image file types
            image_path = os.path.join(input_folder, filename)
            # Load the image
            image = cv2.imread(image_path)

            # Add realistic damage
            damaged_image, damaged_image_mask = add_realistic_damage(image, num_scratches, num_patches)

            # Optionally, add dust and grain for a vintage effect
            #damaged_image_with_grain = add_dust_and_grain(damaged_image, 0.15)

            # Save the processed image to the output folder
            output_path = os.path.join(output_folder, filename)
            output_path1 = os.path.join(output_folder1, filename)
            
            cv2.imwrite(output_path, damaged_image)
            cv2.imwrite(output_path1, damaged_image_mask)

In [16]:
input_folder = r'C:\Users\DELL\OneDrive\Desktop\DIP\europeana_images'  # Replace with your input folder path
output_folder = r'C:\Users\DELL\OneDrive\Desktop\DIP\images'  # Replace with your output folder path
output_folder1 = r'C:\Users\DELL\OneDrive\Desktop\DIP\mask'

process_images(input_folder, output_folder, output_folder1)

Processing images: 100%|██████████████████████████████████████████████████████████| 813/813 [00:24<00:00, 33.71image/s]
